# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import json
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bredasdorp,-34.5322,20.0403,16.22,77,100,3.24,ZA,1704606190
1,1,adamstown,-25.0660,-130.1015,22.48,76,100,7.10,PN,1704606190
2,2,iskateley,67.6803,53.1512,-16.10,95,16,3.13,RU,1704605616
3,3,east london,-33.0153,27.9116,20.52,83,0,5.14,ZA,1704606190
4,4,tiksi,71.6872,128.8694,-34.15,98,42,3.08,RU,1704606191


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    c="Humidity",
    hover_cols=["City"],
    colormap="plasma",
    title="City Map with Humidity",
    xlabel="Longitude",
    ylabel="Latitude",
    size="Humidity",
    frame_width=600,
    frame_height=400,
    tiles="OSM",
    geo=True
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
fit_city_data_df = city_data_df[
    (city_data_df["Max Temp"] >= 20) & (city_data_df["Max Temp"] <= 30) &
    (city_data_df["Wind Speed"] < 5) & (city_data_df["Cloudiness"] < 20)
]
# Drop any rows with null values
drop_city_data = fit_city_data_df.dropna()

# Display sample data
drop_city_data.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,port elizabeth,-33.9180,25.5701,21.29,78,0,3.60,ZA,1704605606
46,46,takoradi,4.8845,-1.7554,26.97,85,14,3.14,GH,1704605643
70,70,sao jose da coroa grande,-8.8978,-35.1478,25.58,77,12,3.57,BR,1704605635
81,81,lihue,21.9789,-159.3672,22.77,78,0,1.54,US,1704606211
82,82,cartagena,10.3997,-75.5144,25.79,89,0,1.54,CO,1704606211


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,bredasdorp,-34.5322,20.0403,16.22,77,100,3.24,ZA,1704606190,
1,1,adamstown,-25.0660,-130.1015,22.48,76,100,7.10,PN,1704606190,
2,2,iskateley,67.6803,53.1512,-16.10,95,16,3.13,RU,1704605616,
3,3,east london,-33.0153,27.9116,20.52,83,0,5.14,ZA,1704606190,
4,4,tiksi,71.6872,128.8694,-34.15,98,42,3.08,RU,1704606191,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": radius,
    "categories": "tourism,accommodation",
    "apiKey": geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bredasdorp - nearest hotel: Victoria Hotel
adamstown - nearest hotel: No hotel found
iskateley - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
tiksi - nearest hotel: Арктика
campos belos - nearest hotel: No hotel found
port elizabeth - nearest hotel: Waterford Hotel
jamestown - nearest hotel: Robert H. Jackson Center
blackmans bay - nearest hotel: No hotel found
morondava - nearest hotel: Hotel Sharon Menabe
thompson - nearest hotel: Thompson Inn
gisborne - nearest hotel: No hotel found
grytviken - nearest hotel: Guano Factory
summerside - nearest hotel: Comme Chez Vous
tolanaro - nearest hotel: Hôtel Mahavokey
anna regina - nearest hotel: Damon Monument
olonkinbyen - nearest hotel: Moon surface viewpoint
lompoc - nearest hotel: Alano Club
chonchi - nearest hotel: Centenario
sirjan - nearest hotel: خانه معلم سیرجان
margaret river - nearest hotel: No hotel found
nova vicosa - nearest hotel: Baleia Jubarte
yinchuan - nearest hotel: 鼓楼
alb

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,bredasdorp,-34.5322,20.0403,16.22,77,100,3.24,ZA,1704606190,Victoria Hotel
1,1,adamstown,-25.0660,-130.1015,22.48,76,100,7.10,PN,1704606190,No hotel found
2,2,iskateley,67.6803,53.1512,-16.10,95,16,3.13,RU,1704605616,No hotel found
3,3,east london,-33.0153,27.9116,20.52,83,0,5.14,ZA,1704606190,No hotel found
4,4,tiksi,71.6872,128.8694,-34.15,98,42,3.08,RU,1704606191,Арктика
...,...,...,...,...,...,...,...,...,...,...,...
580,580,sao jose do egito,-7.4789,-37.2744,21.57,75,16,2.30,BR,1704606196,No hotel found
581,581,kapuskasing,49.4169,-82.4331,-12.95,85,100,4.12,CA,1704606340,Advantage Motel
582,582,semera,11.5010,41.2010,22.79,47,82,4.53,ET,1704606340,No hotel found
583,583,roses,42.2620,3.1769,9.50,58,29,7.60,ES,1704606340,Ramblamar


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# Configure the map plot
hotel_city_map = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    c="Humidity",
    hover_cols=["City", "Country", "Hotel Name"],
    colormap="plasma",
    title="City Map with Humidity and Hotel Information",
    xlabel="Longitude",
    ylabel="Latitude",
    size="Humidity",
    frame_width=600,
    frame_height=400,
    tiles="OSM",
    geo=True
)

# Display the map
hotel_city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)